In [1]:
import importlib
import subprocess

import Advanced_SQL as asql
import duckdb
import SQL as sql

_ = importlib.reload(asql)
_ = importlib.reload(sql)

Use this cell to test any sql queries. This can be done in one of the following ways:

1. Fill in the `qry` within the """ """.
2. Call the question functions that you have already filled in from the SQL.py and Advanced_SQL.py files. An example of question_1 being called from the SQL.py file is currently commented out.

This cell can be copied if you wish to run multiple queries.


In [17]:
_ = importlib.reload(asql)
_ = importlib.reload(sql)

qry = """  
    create or replace table financing as
    -- we first need to dedupe the data as both tables contain 
    -- duplicate customer ids
    with customers_deduped as (
        select *
        from customers
        qualify row_number() over (partition by customerid order by customerid) = 1
    ),
    loans_deduped as (
        select *
        from loans
        qualify row_number() over (partition by customerid order by customerid) = 1
    ),
    -- credit table also contains customerid duplicates and needs to be deduped
    credit_deduped as (
        select *
        from credit
        qualify row_number() over (partition by customerid order by customerid) = 1
    )
    select 
        customers_deduped.customerid as CustomerID
        ,customers_deduped.income as Income
        ,loans_deduped.loanamount as LoanAmount
        ,loans_deduped.loanterm as LoanTerm
        ,loans_deduped.interestrate as InterestRate
        ,loans_deduped.approvalstatus as ApprovalStatus
        ,credit_deduped.creditscore as CreditScore
    from customers_deduped
    left join loans_deduped
        on customers_deduped.customerid = loans_deduped.customerid
    left join credit_deduped
        on customers_deduped.customerid = credit_deduped.customerid
"""

qry = asql.question_7()


with duckdb.connect("database/loan.db") as cursor:
    result_set = cursor.execute(qry).df()

result_set

,CustomerId,Age,CorrectedAge,Gender,AgeCategory,Rank
0,244,78,37,Female,Adult,0
1,321,39,56,Female,Adult,0
2,382,80,55,Male,Adult,0
3,329,64,33,Male,Adult,0
4,995,41,33,Female,Adult,1
...,...,...,...,...,...,...
455,122,59,58,Male,Adult,12
456,888,72,41,Female,Adult,12
457,200,78,42,Male,Adult,12
458,16,41,44,Male,Adult,12


Should you break/incorrectly update any database tables, the database can be reset by running the following cell


In [62]:
load_script_path = "database/database_load.py"
subprocess.run(["python3", load_script_path])

CompletedProcess(args=['python3', 'database/database_load.py'], returncode=0)